In [1]:
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [2]:
def feature_selection_with_gradient_boosting(data: pd.DataFrame,
                                             target: str,
                                             drop_threshold: float = 0.75,
                                             max_iter_mice: int = 12,
                                             n_splits: int = 5,
                                             random_state: int = 42,
                                             gb_params: dict = None):
    """
    Perform feature selection based on Gradient Boosting feature importances.
    
    Process:
        1. Drop rows with missing ratio > drop_threshold.
        2. MICE imputation for missing values.
        3. Train a Gradient Boosting model using all features to obtain feature importances.
        4. Sort features by importance in descending order.
        5. For top X% of features (X from 10% to 50% with step=5%), re-train the model
           and evaluate performance via cross-validation.
        6. Print results for each subset of features.

    :param data: Input dataframe (includes features + target column).
    :param target: The name of the target column.
    :param drop_threshold: If row's missing ratio > drop_threshold, drop it. Default=0.95.
    :param max_iter_mice: Number of MICE iteration, default=12.
    :param n_splits: Number of folds for cross-validation, default=5.
    :param random_state: Seed for reproducibility, default=42.
    :param gb_params: A dict of hyperparameters for GradientBoostingClassifier, 
                      e.g., {"n_estimators":300, "learning_rate":0.05, "max_depth":4}.
                      If None, we use a default config.
    :return: None (prints out the performance results of different feature subsets).
    """

    # 1. Drop rows with too many missing values
    row_missing_ratio = data.isnull().sum(axis=1) / data.shape[1]
    data_cleaned = data.loc[row_missing_ratio <= drop_threshold].copy()
    
    # 2. Separate target and features
    y = data_cleaned[target]
    X = data_cleaned.drop(columns=[target])
    feature_names = X.columns

    # 3. MICE imputation
    imputer = IterativeImputer(max_iter=max_iter_mice, random_state=random_state)
    X_imputed = imputer.fit_transform(X)

    # 4. Train a Gradient Boosting model to get feature importances
    if gb_params is None:
        gb_params = {
            "n_estimators": 100,
            "learning_rate": 0.1,
            "max_depth": 3,
            "random_state": random_state
        }
    gb_clf = GradientBoostingClassifier(**gb_params)
    gb_clf.fit(X_imputed, y)

    # Get feature importances and sort
    importances = gb_clf.feature_importances_
    feat_imp_df = pd.DataFrame({
        "Feature": feature_names,
        "Importance": importances
    }).sort_values(by="Importance", ascending=False).reset_index(drop=True)

    # For convenience, get sorted feature names
    sorted_features = feat_imp_df["Feature"].tolist()

    # 5. For coverage from 10% to 50% in increments of 5%, select top features and re-train
    n_total_features = len(feature_names)
    coverage_list = range(10, 81, 5)  # 10%, 15%, 20%, ..., 50%

    results = []
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    for coverage in coverage_list:
        # Compute how many features to keep
        n_keep = max(1, int(np.ceil(n_total_features * (coverage / 100.0))))
        
        # Select top n_keep features
        selected_feats = sorted_features[:n_keep]
        
        # Build a subset of X_imputed with only those features
        selected_indices = [feature_names.get_loc(feat) for feat in selected_feats]
        X_sub = X_imputed[:, selected_indices]

        # Re-train Gradient Boosting on these top features using cross-validation
        model = GradientBoostingClassifier(**gb_params)
        scores = cross_val_score(model, X_sub, y, cv=cv, scoring="accuracy", n_jobs=-1)
        
        mean_acc = scores.mean()
        std_acc = scores.std()

        results.append({
            "Coverage(%)": coverage,
            "Num_Features": n_keep,
            "Mean_Accuracy": mean_acc,
            "Std_Accuracy": std_acc
        })

    # 6. Print feature importance + results
    print("\n=== Feature Importances (All Features) ===")
    print(feat_imp_df.to_string(index=False))

    print("\n=== Feature Selection Results (Top X% of Features) ===")
    results_df = pd.DataFrame(results)
    print(results_df.to_string(index=False))

In [3]:
def main():
    """
    Main function to demonstrate feature_selection_with_gradient_boosting usage.
    """

    # Example: read data (please modify the path as necessary)
    data = pd.read_csv(r"../data/final/nicu_30.csv")

    # Suppose your target column name is 'is_infected'
    target_column = "is_infected"

    # Example gradient boosting hyperparameters from prior knowledge or tuning
    gb_params = {
        "n_estimators": 50,
        "learning_rate": 0.05,
        "max_depth": 5,
        "subsample": 0.6, 
        "random_state": 42
    }

    # Call the feature selection function
    feature_selection_with_gradient_boosting(
        data=data,
        target=target_column,
        drop_threshold=0.75,
        max_iter_mice=12,
        n_splits=5,
        random_state=42,
        gb_params=gb_params
    )

In [4]:
if __name__ == "__main__":
    main()


=== Feature Importances (All Features) ===
                     Feature  Importance
          Lymphocytes_min_30    0.108206
          Lymphocytes_max_30    0.085953
         Lymphocytes_mean_30    0.075930
                 SaO2_min_30    0.060169
      BP Cuff [Mean]_mean_30    0.028227
  BP Cuff [Systolic]_mean_30    0.024705
                 SaO2_max_30    0.024296
     Temp Axilary [F]_max_30    0.021776
                SaO2_mean_30    0.021507
           Heart Rate_max_30    0.019217
                  SUBJECT_ID    0.018593
      Temp/Iso/Warmer_min_30    0.017325
           Glucometer_min_30    0.016725
   BP Cuff [Systolic]_min_30    0.016677
            Resp Rate_max_30    0.015651
                 MCV_mean_30    0.015099
      Red Blood Cells_max_30    0.015062
         Neutrophils_mean_30    0.014619
      Temp/Iso/Warmer_max_30    0.014610
           Glucometer_max_30    0.014503
    White Blood Cells_min_30    0.014376
      Red Blood Cells_min_30    0.013511
       Platel